In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
import datetime as dt
from datetime import datetime
import numpy as np 
from nltk.stem.snowball import SnowballStemmer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', -1)

days = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']



df = pd.read_csv('tweets_df.csv')



df['Date'] = df['Date'].astype('str')
df['Time'] = df['Time'].astype('str')
df['Date'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df = df.drop('Time', 1)
df = df.assign(Date=df.Date.dt.round('H'))




start_date = '2020-04-27'
end_date = '2020-11-03'
dates = (df['Date'] > start_date) & (df['Date'] <= end_date)
df = df.loc[dates]



# for i in range(len(days)): 
#     df['Text'] = df['Text'].str.rstrip(days[i])

# for i in range(len(months)): 
#     df['Text'] = df['Text'].str.rstrip(months[i])

def clean(text):
    text = re.sub('\\b(monday|tuesday|wedenesday|thursday|friday|saturday|sunday)\\W','', text)
    text = re.sub('\\b(january|february|march|april|may|june|july|august|september|october|november|december)\\W','', text)
    return text

df['Text'] = df['Text'].apply(clean)

stop_words = set(stopwords.words('english'))
df['Text'] = df['Text'].apply(lambda x: [item[1:-1] for item in x[1:-1].replace(" ", "").split(",") if item[1:-1] not in stop_words])
stemmer = SnowballStemmer('english')
df['Text'] = df['Text'].apply(lambda x: [stemmer.stem(i) for i in x])
df['Text'] = df['Text'].str.join(' ')



price_evolution = pd.read_excel('brent_price.xlsx')
price_evolution['Date'] = pd.to_datetime(price_evolution['Date'])
price_evolution.set_index('Date', inplace = True)


train_df = pd.merge(df, price_evolution, on = 'Date')
train_df = train_df.set_index('Date')
train_df = train_df.sort_values(by = 'Date', ascending = True)
train_df = train_df.groupby(train_df.index.floor('H')).agg({'Text': ''.join, 'Price': np.mean, 'Variation': np.mean,})
# train_df['Delta'] = train_df.Price.diff(-1)





In [2]:
sigma = train_df['Variation'].std()

In [3]:
train_df

,Text,Price,Variation
Date,,,
2020-04-27 07:00:00,move end-game.” next chapter oil crisis: industri shut down. via pork belli era pandem eating. (and butter price fall restaur closed) n'via,32.06,-0.006508
2020-04-27 08:00:00,pandem econom consequ rescu disarma effort upcom confer non-prolifer treaty. find read energi intellig here:,31.55,-0.015908
2020-04-27 09:00:00,china client said lose $ billion oil bet join expert mike sultan energi discuss impact current market condit lng price trade take deeper look long-term demand. regist here: publish articl co-author &amp; . pric cost cut drive recoveri oil industry. articl build find earlier simon-kucher/ric univers study.nread articl here:,31.88,0.010460
2020-04-27 11:00:00,plan cut product start st opec oottpart opec plusopec plus state follow oman lead provid detail plan stick new quota transpar oott,32.26,0.006552
2020-04-27 12:00:00,$gm suspend quarter cash dividen share buyback,32.30,0.001240
2020-04-27 14:00:00,it anoth crazi day oott opec,32.11,-0.000933
2020-04-27 17:00:00,anyth offset come oil energi cliff.nnani altern noth fossil fuel derivatives.,31.91,0.001255
2020-04-27 18:00:00,revis $. billion alaska deal oil price crash,31.78,-0.004074
2020-04-27 20:00:00,told month ago that:nnunemploy jump nnfed interest rate fall . nnpmi dive nngold rip nnstock volatil triplesnnoil crash nn....and ye past yea stock flat.nnwould believ,32.27,0.011916


In [4]:
train_df['Sentiment'] = ''

for i in range(len(train_df.index)):
    if train_df.iloc[i, 2] > 0.5 * sigma:
        train_df['Sentiment'][i] = 1
    elif train_df.iloc[i, 2] < -0.5 * sigma:
         train_df['Sentiment'][i] = -1
    elif -0.5 * sigma <= train_df.iloc[i, 2] <= 0.5 * sigma:
        train_df['Sentiment'][i] = 0

/Users/gioel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/gioel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/gioel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [5]:
def clean(text):
    text = re.sub(r'[^\w\s]','', text)
    text = re.sub('/(^| ).( |$)/','', text)
    text = re.sub(r'@[A-Za-z0–9]+', '', text)
    text = re.sub(r'$', '', text)
    return text

train_df['Text'] = train_df['Text'].apply(clean)


In [6]:
days = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']

for i in range(len(days)): 
    train_df['Text'] = train_df['Text'].str.rstrip(days[i])

for i in range(len(months)): 
    train_df['Text'] = train_df['Text'].str.rstrip(months[i])

In [7]:
train_df.to_csv('train_df.csv')

In [8]:
train_df[train_df['Text'].str.contains('june')]

,Text,Price,Variation,Sentiment
Date,,,,
2020-04-30 18:00:00,petroleum qp slash budget plan job cut help deal slump energi industri caus coronavirus pandemi accord letter compani manag seen energi minist alexand novak wednesday said countri oil output could fall yea interfax news agenc report repres first annual declin sinc forecast latest with close up n q brentwti volatil expect junen h low shigh s brentwti av higher exitn mid s bot handl brentnnrestart keynnoott,32.39,-0.017294,-1
2020-06-16 15:00:00,aramco push back offshor expans timelin year aramcoacceler oil output cut opec midjunennaccord figur russia produc mln barrel oil per day at quota mln barrel per day reduc product mln barrel per daynoottnoffici announc aramco sabic deal tomorrow oott aramcoquick look prices brent back,42.54,0.001648,0
2020-07-08 11:00:00,cut top million bd junennread top stori opecplus amp oil market free,43.65,-0.002742,-1
2020-10-29 14:00:00,time rollov current opec plus cut enough prop prices oottprice collaps month low lagard commentdrop oott opecbank gold sale account ton bullion third quarter nnsell driven uzbekistan turk russia central bank also post first quarter sale years nnvia time rollov current opec plus cut enough prop prices oottuse flexibl pepp addressrun esf back yesterday low tri form basennplan todayabov trigger run otherwi next support key resist zone caus june selloff key support end septemb sellofftime rollov current opec plus cut enough prop prices,37.76,-0.000529,0
